In [1]:
import os
import re
import numpy as np
import pandas as pd

In [2]:
data_dir = 'benchmarking-results'

In [3]:
os.listdir(data_dir)

['base-clang-cBench-benchmarks.log',
 'test-policy-cpp-perf-benchmarks.log',
 'test-policy-cBench-benchmarks.log',
 'old-cpp-perf-benchmarks.log',
 'base-clang-cpp-perf-benchmarks.log',
 'mlgo-cBench-benchmarks.log',
 'mlgo-cpp-perf-benchmarks.log',
 'old-cBench-benchmarks.log',
 'policy-2-cpp-perf-benchmarks.log',
 'mlgo-2-cpp-perf-benchmarks.log']

In [4]:
prefixes = [
#     'base-clang',
#     'test-policy',
#     'mlgo'
    'base-clang',
    'policy-2',
    'mlgo-2'
]

labels = [
    'base',
    'policy',
    'mlgo'
]

benchmarks = [
    'cpp-perf-benchmarks',
#     'cBench-benchmarks'
]

In [5]:
def calc_adjusted_mean(values):
    min_val = min(values)
    max_val = max(values)
    
    return (sum(values) - (max_val + min_val)) / 3

def parse_lines(lines):
    results = {}

    key = None
    for line in lines:
        if '##' in line:
            continue
        elif 'UTC' in line:
            continue
        elif key is not None and 'real' in line and 'm' in line:
            if key not in results:
                results[key] = []
            match = re.search(r'(\d+)m?(\d+(\.\d+)?)', line)
            if match:
                groups = match.groups()
                minutes = float(groups[0])
                seconds = float(groups[1])
                results[key].append((minutes * 60) + seconds)
        elif len(line) > 0 and 'real' not in line and 'user' not in line and 'sys' not in line:
            #print('updating key ->', key)
            key = line
    
    results_adj = {
        "file": [],
        "mean_runtime": [],
        "variance": [],
        "raw_measurements": []
    }
    for file in results:
        values = results[file]
        mean = calc_adjusted_mean(values)
        var = np.var(values)
        
        results_adj["file"].append(file)
        results_adj["mean_runtime"].append(mean)
        results_adj["variance"].append(var)
        results_adj["raw_measurements"].append(values)
    
    return pd.DataFrame.from_dict(results_adj)

In [6]:
prefix = prefixes[0]
benchmark = benchmarks[0]

lines = []
with open(os.path.join(data_dir, prefix+'-'+benchmark+'.log'), 'r') as datafile:
    for line in datafile:
        lines.append(line.replace('\t', ' ').replace('\n', ''))

In [7]:
parse_lines(lines).head()

,file,mean_runtime,variance,raw_measurements
0,loop removal,10.999000,0.000991,"[10.969, 10.972, 11.054, 11.007, 11.018]"
1,locales,106.503000,7.966743,"[106.82300000000001, 113.432, 106.387, 106.299..."
2,rotate bits,122.935667,0.010828,"[122.879, 123.161, 122.997, 122.915, 122.895]"
3,scalar replacement structs,95.887333,0.014584,"[95.813, 96.024, 96.092, 95.825, 95.809]"


In [9]:
# prefix = prefixes[0]
# benchmark = benchmarks[1]

# lines = []
# with open(os.path.join(data_dir, prefix+'-'+benchmark+'.log'), 'r') as datafile:
#     for line in datafile:
#         lines.append(line.replace('\t', ' ').replace('\n', ''))

In [10]:
# parse_lines(lines).head()

In [11]:
from itertools import product

In [12]:
df = pd.DataFrame(columns=[
    'file',
    'mean_runtime',
    'variance'
])
for prefix, benchmark in product(prefixes, benchmarks):
    filepath = os.path.join(data_dir, prefix+'-'+benchmark+'.log')
    print(filepath)
    
    lines = []
    with open(filepath, 'r') as datafile:
        for line in datafile:
            lines.append(line.replace('\t', ' ').replace('\n', ''))
    
    file_df = parse_lines(lines)
    file_df['set'] = 'cpp-perf' if 'cpp-perf' in benchmark else 'cBench'
    file_df['agent'] = 'base' if 'base' in prefix else 'policy' if 'policy' in prefix else 'mlgo'
    
    df = pd.concat((df, file_df), axis=0, ignore_index=True)
df.head()

benchmarking-results/base-clang-cpp-perf-benchmarks.log
benchmarking-results/policy-2-cpp-perf-benchmarks.log
benchmarking-results/mlgo-2-cpp-perf-benchmarks.log


,file,mean_runtime,variance,raw_measurements,set,agent
0,loop removal,10.999000,0.000991,"[10.969, 10.972, 11.054, 11.007, 11.018]",cpp-perf,base
1,locales,106.503000,7.966743,"[106.82300000000001, 113.432, 106.387, 106.299...",cpp-perf,base
2,rotate bits,122.935667,0.010828,"[122.879, 123.161, 122.997, 122.915, 122.895]",cpp-perf,base
3,scalar replacement structs,95.887333,0.014584,"[95.813, 96.024, 96.092, 95.825, 95.809]",cpp-perf,base
4,loop removal,240.072000,0.000170,"[240.082, 240.062, 240.054, 240.072, 240.09]",cpp-perf,policy


In [13]:
df[df.set == 'cpp-perf'][df.file != 'machine']\
    .groupby(['file', 'agent'])\
    [['mean_runtime', 'variance']]\
    .last()\
    .apply(lambda x: round(x, 4))\
    .to_latex()

/tmp/ipykernel_970203/760010953.py:6: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  .to_latex()


'\\begin{tabular}{llrr}\n\\toprule\n                           &        &  mean\\_runtime &  variance \\\\\nfile & agent &               &           \\\\\n\\midrule\nlocales & base &      106.5030 &    7.9667 \\\\\n                           & mlgo &      104.1613 &    0.1510 \\\\\n                           & policy &      104.3913 &    0.7208 \\\\\nloop removal & base &       10.9990 &    0.0010 \\\\\n                           & mlgo &      240.0443 &    0.0001 \\\\\n                           & policy &      240.0720 &    0.0002 \\\\\nrotate bits & base &      122.9357 &    0.0108 \\\\\n                           & mlgo &      240.0523 &    0.0001 \\\\\n                           & policy &      240.0767 &    0.0007 \\\\\nscalar replacement structs & base &       95.8873 &    0.0146 \\\\\n                           & mlgo &      240.0547 &    0.0002 \\\\\n                           & policy &      240.0723 &    0.0001 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [ ]:
mlgoperf_cbench_files = [
    'automotive_bitcount',
    'automotive_qsort1',
    'automotive_susan_c',
    'automotive_susan_e',
    'automotive_susan_s',
    'bzip2d',
    'bzip2e',
    'consumer_jpeg_c',
    'consumer_jpeg_d',
    'consumer_lame',
    'consumer_tiff2bw',
    'consumer_tiff2rgba',
    'consumer_tiffdither',
    'consumer_tiffmedian',
    'network_dijkstra',
    'network_patricia',
    'office_stringsearch1',
    'security_blowfish_d',
    'security_blowfish_e',
    'security_rijndael_d',
    'security_rijndael_e',
    'security_sha',
    'telecom_adpcm_c',
    'telecom_adpcm_d',
    'telecom_CRC32',
]

In [ ]:
df\
    [df.set == 'cBench']\
    [~df.file.str.contains('a.out')]\
    [df.file.isin(mlgoperf_cbench_files)]\
    .groupby(['file', 'agent'])\
    [['mean_runtime', 'variance']]\
    .last()\
    .apply(lambda x: round(x, 4))\
    .to_latex()